In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP project/cleaned_v2.xlsx", index_col=0)

labels = pd.read_excel(r"/content/drive/MyDrive/Colab Notebooks/NLP project/processed/claims_russia.xlsx", index_col=0)

labels = list(labels.index.to_numpy())
labels

['Russia interfered in the US election',
 'Russia threatens to use nuclear weapons',
 'Russia withdraws from Kherson',
 'Russia rejects the conviction for the downing of flight MH17',
 'Russia denies responsibility for the MH17 disaster',
 "Russia's economy is in recession",
 'The missile that hit Poland was fired by Russia',
 'Russia has been torturing Ukrainians',
 'The Russia-Ukraine war is an invasion',
 'Russia strikes Ukrainian nuclear power plant',
 'Vladimir Putin is a illegitimate dictator',
 'Europe relies on Russian energy',
 'Russia is commiting war crimes in Ukraine',
 'Russia fires missile barrage at Ukraine',
 'Brittney Griner moved to penal colony',
 'Russian convicts are being enlisted to the military',
 'Three men convicted for the downing of MH17',
 'Russian men are escaping from conscription',
 'Russia is in a recession',
 'Russia is at fault for the missile hitting Poland',
 'Russia is torturing Ukrainians',
 'Finland is building a fence on the Russian border',
 'R

In [3]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"DEVICE: {device} \n---------------------------------------------------------\n\n")

model_name = "MoritzLaurer/DeBERTa-v3-large-mnli-fever-anli-ling-wanli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)

premise = "I first thought that I liked the movie, but upon second thought it was actually disappointing."
hypothesis = "The movie was not good."

input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.5 MB/s eta 0:00:00
DEVICE: cuda 
---------------------------------------------------------




{'entailment': 83.6, 'neutral': 16.2, 'contradiction': 0.3}


In [ ]:
article = list(df.cleaned_content.loc[df.uid == 'russia_260'].to_numpy())[0]

In [4]:
articles = dict(zip(df.uid.loc[df.topic == 'russia'], df.cleaned_content.loc[df.topic == 'russia']))



In [16]:
import os.path
from tqdm import tqdm


file_path = "/content/drive/MyDrive/Colab Notebooks/NLP project/generated claim ver/russia_claims.xlsx"

out = pd.DataFrame(columns=['claim', 'high_supporting', 'high_rejecting', 'med_supporting', 'med_rejecting', 'supporting_count', 'neutral_count', 'rejecting_count'])

progress_bar = tqdm(range(len(articles.keys()) * len(labels)))


for claim in labels:
    new_row = {'claim' : claim, 'high_supporting' : [], 'high_rejecting' : [], 'med_supporting' : [], 'med_rejecting' : [], 'supporting_count' : 0, 'neutral_count' : 0,
               'rejecting_count' : 0}
    out = out.append(new_row, ignore_index=True)
    print(claim)
    support_count, neutral_count, rejecting_count = 0,0,0
    for uid, article in articles.items():
        premise = article
        hypothesis = claim

        input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
        output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
        prediction = torch.softmax(output["logits"][0], -1).tolist()
        label_names = ["entailment", "neutral", "contradiction"]
        prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
        try:
            if prediction[max(prediction, key=prediction.get)] > 90.0:
                if max(prediction, key=prediction.get) == 'neutral':
                    neutral_count += 1
                elif max(prediction, key=prediction.get) == 'entailment':
                    support_count += 1
                    results = out['high_supporting'].loc[out['claim'] == claim].item()
                    results.append(uid)
                elif max(prediction, key=prediction.get) == 'contradiction':
                    rejecting_count += 1
                    results = out['high_rejecting'].loc[out['claim'] == claim].item()
                    results.append(uid)

            elif prediction[max(prediction, key=prediction.get)] > 60.0:
                if max(prediction, key=prediction.get) == 'neutral':
                    neutral_count += 1
                elif max(prediction, key=prediction.get) == 'entailment':
                    support_count += 1
                    results = out['med_supporting'].loc[out['claim'] == claim].item()
                    results.append(uid)
                elif max(prediction, key=prediction.get) == 'contradiction':
                    rejecting_count += 1
                    results = out['med_rejecting'].loc[out['claim'] == claim].item()
                    results.append(uid)
        finally:
            progress_bar.update()
            progress_bar.display()
    out['supporting_count'].loc[out['claim'] == claim] = support_count
    out['neutral_count'].loc[out['claim'] == claim] = neutral_count
    out['rejecting_count'].loc[out['claim'] == claim] = rejecting_count
    

    out.to_excel(file_path)

  0%|          | 3/23108 [00:00<25:42, 14.97it/s]

Russia interfered in the US election


  2%|▏         | 439/23108 [00:30<25:14, 14.96it/s]

Russia threatens to use nuclear weapons


  4%|▍         | 875/23108 [01:01<25:02, 14.80it/s]

...

 96%|█████████▌| 22239/23108 [25:42<00:59, 14.57it/s]

Russia planned to attack Japan


 98%|█████████▊| 22675/23108 [26:12<00:29, 14.63it/s]

Russia fires mock nuclear warhead


100%|██████████| 23108/23108 [26:42<00:00, 14.76it/s]

In [15]:
out['supporting_count'].loc[out['claim'] == claim]

52    1
Name: supporting_count, dtype: object

In [ ]:
import operator
import os.path
from tqdm import tqdm


file_path = "/content/drive/MyDrive/Colab Notebooks/NLP project/generated claim ver/US elections_claims.xlsx"

#if os.path.isfile(file_path):
#    out = pd.read_excel(file_path, index_col=0)
#else:
out = pd.DataFrame(columns=['claim', 'supporting', 'rejecting'])

progress_bar = tqdm(range(len(articles.keys()) * len(labels)))

for claim in labels:
    new_row = {'claim' : claim, 'supporting' : [], 'rejecting' : []}
    out = out.append(new_row, ignore_index=True)
    #print(claim)
    for uid, article in articles.items():
        premise = article
        hypothesis = claim

        input = tokenizer(premise, hypothesis, truncation=True, return_tensors="pt")
        output = model(input["input_ids"].to(device))  # device = "cuda:0" or "cpu"
        prediction = torch.softmax(output["logits"][0], -1).tolist()
        label_names = ["entailment", "neutral", "contradiction"]
        prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
        if max(prediction, key=prediction.get) == 'entailment' and prediction[max(prediction, key=prediction.get)] > 90.0:
            results = out.supporting.loc[out['claim'] == claim].item()
            results.append(uid)
            #print(uid)
            #print(prediction)
        elif max(prediction, key=prediction.get) == 'contradiction' and prediction[max(prediction, key=prediction.get)] > 90.0:
            results = out.rejecting.loc[out['claim'] == claim].item()
            results.append(uid)
            #print(uid)
            #print(prediction)
        progress_bar.update()
        progress_bar.display()
    out.to_excel(file_path)

100%|██████████| 55440/55440 [1:02:22<00:00, 16.62it/s]

In [ ]:
import time
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor
def worker(thread_number):
    for i in tqdm(range(100), desc = f'thread {thread_number}'):
        time.sleep(0.05*thread_number)
        
if __name__ == '__main__':
    thread_list = list(range(1,4))
    with ThreadPoolExecutor() as p:
        p.map(worker, thread_list)

thread 2:   0%|          | 0/100 [00:00<?, ?it/s]

thread 1:   0%|          | 0/100 [00:00<?, ?it/s]

thread 3:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
import operator
prediction[max(prediction.items(),key=operator.itemgetter(1))[0]]

43.4

In [ ]:
out.supporting.loc[out['claim'] == claim].item()

[]

In [ ]:
tuple(tup + (uid,))

TypeError: can only concatenate str (not "tuple") to str

In [ ]:
tup = out.rejecting.loc[out['claim'] == claim][0]
t = ()
t = (tup,) + (uid,)

In [ ]:
t

((), 'russia_9')